In [35]:
from dataReadFromDisk import load_df_from_csv,process_audio_and_beatmap_for_model_all_and_save,load_all_beatmaps_from_df,process_audio_and_beatmap_for_model_single

In [36]:
df = load_df_from_csv()

DataFrame loaded from: E:\\Notebooks\\osu\processed-metadata\processed-metadata.csv


In [37]:
df = df[:10]
# df.iloc[7]
# df = df[df["audio"] == "0013ddfd8bd55fdccc0b253e313b7a60"]
# df

In [38]:
# testa,testb = process_audio_and_beatmap_for_model_single(df.iloc[0])
# testa.shape
# # for h in testb:
# #     print(h)

In [39]:
dataset,num_of_batches,processed_audio_list,processed_beatmap_list = process_audio_and_beatmap_for_model_all_and_save(df)

In [40]:
num_of_batches

10

In [41]:
for h in processed_audio_list[20][0:10]:

    print(h)

IndexError: list index out of range

In [ ]:
from makeBeatmap import make_beatmap_from_output
make_beatmap_from_output(df.iloc[20],processed_beatmap_list[20])

File 'aiOutput\output.osu' has been created.


In [ ]:
# for input_data, output_data in dataset.take(1):
#     print("First input:")
#     print(input_data.numpy()[0][0])
#     print("First output:")
#     print(output_data.numpy()[0][0])

In [ ]:
# import numpy as np
# import tensorflow as tf

# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(None, 142)),  # Input layer for variable length sequences with 142 features
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     # tf.keras.layers.Dense(64, activation='relu'),  # Optional dense layer
#     tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
# ])

# model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# # Training the model
# model.fit(dataset, epochs=10, steps_per_epoch=num_of_batches) # instead of 50 make it number of songs in final processed data

In [ ]:
# import tensorflow as tf

# # Check if GPU is available
# gpus = tf.config.experimental.list_physical_devices()
# gpus

In [ ]:
# print(tf.config.list_physical_devices('GPU'))

In [ ]:
# tf.__version__

In [59]:
import tensorflow as tf

# # Ensure TensorFlow is set up to use the GPU
# physical_devices = tf.config.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     print("GPU is available and configured.")
# else:
#     print("GPU is not available.")

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

def custom_loss(y_true, y_pred):
    hit_weight = 5.0  # Higher weight for hit objects
    no_hit_weight = 1.0  # Lower weight for no hit objects (zeros)
    
    # Compute the Mean Squared Error (MSE) loss
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)  # Shape: (batch_size, time_steps, output_dim)
    
    # Create a weight matrix based on the true values (broadcast across batch and time steps)
    weights = tf.where(tf.not_equal(y_true, 0), hit_weight, no_hit_weight)  # Shape: (batch_size, time_steps, output_dim)
    
    # Ensure the weights have the same shape as the loss
    weights = tf.cast(weights, tf.float32)
    
    # Apply the weights to the loss (element-wise multiplication)
    weighted_loss = tf.multiply(loss, weights)  # Element-wise multiplication
    
    # Return the mean loss across the batch and time steps
    return tf.reduce_mean(weighted_loss)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 142)),  # Input layer for variable-length sequences with 142 features
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dense(64),  # Optional dense layer
    tf.keras.layers.Dense(35, activation='relu')  # Output layer, size 35 to match your output data dimension
])

# Compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=custom_loss)

# Assuming `dataset` is already prepared and `num_of_batches` is defined
# Training the model

dataset = dataset.repeat()
with tf.device('/gpu:0'):
    model.fit(dataset, epochs=2, steps_per_epoch=num_of_batches)
# After running, you can monitor GPU usage using `nvidia-smi` from the command line


Num GPUs Available:  1
Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node 'custom_loss/Mul' defined at (most recent call last):
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\darsh\anaconda3\envs\latestEnv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\darsh\anaco

In [43]:
from dotenv import load_dotenv
import os
#setting data path
load_dotenv()
project_path = os.getenv('PROJECT_PATH')
model.save(f'{project_path}/model_name.h5')

In [44]:
input_data = tf.ragged.constant([processed_audio_list[0]])  # Example input with sequence length 100 and 142 features

# Generate output using the loaded model
output = model.predict(input_data)

1/1 [==============================] - 4s 4s/step


In [45]:
for h in output[0]:
    print(h)

tf.Tensor(
[ 0.03002729    0.08467754    0.2365228    -0.025580412   0.16528599
 -0.059712887   0.54068273   -0.167226      0.0012100553  0.48341137
  0.18154919   -0.049233317   0.31728926   -0.6300643    -0.24230397
  0.41640565    0.16051391   -0.16243678   -0.18474625   -0.064191535
  0.56401134    0.17943339    0.40894824    0.30175197    0.08706388
  0.07891271    0.17673822   -0.15385075    0.37872535   -0.033079185
  0.4052122     0.19094315   -0.12497483    0.16107109    0.42153102  ], shape=(35,), dtype=float32)
tf.Tensor(
[-0.03943896    0.006221083   0.15391745   -0.022836244   0.18503504
 -0.09818435    0.58039033   -0.14086632   -0.0061053     0.54732364
  0.20637551   -0.024939083   0.32395405   -0.73221886   -0.22916563
  0.50178355    0.14630447   -0.15802845   -0.23811395   -0.10381382
  0.6036312     0.22118996    0.519239      0.34188065    0.1071837
  0.0027968418  0.20117085   -0.15494391    0.46295673   -0.005397614
  0.48505092    0.16245383   -0.15532747    0.2

KeyboardInterrupt: 

In [24]:
from makeBeatmap import make_beatmap_from_output
make_beatmap_from_output(df.iloc[0],output[0])

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment